In [50]:
import nest_asyncio
nest_asyncio.apply()

In [51]:
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate

load_dotenv() 
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")

pc = Pinecone(api_key=pinecone_api_key)

embeddings = OllamaEmbeddings(
    base_url="http://localhost:11434", 
    model="nomic-embed-text:latest"
)
llm = ChatOllama(
    base_url="http://localhost:11434", 
    model="llama3.2:latest"
)

if not pc.has_index(pinecone_index_name):
    pc.create_index(
        name=pinecone_index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    
index = pc.Index(pinecone_index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index) 
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3}) 


In [52]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


def debug(input):
    print("INPUT: ", input)
    return input


template = """Answer the users question. Try to answer based on the context below.:
{context}


Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retrieval_chain = {
    "context": (lambda x: x["question"]) | retriever,
    "question": lambda x: x["question"],
} | prompt
complete_chain = RunnableLambda(debug) | retrieval_chain | llm | StrOutputParser()

In [53]:
# %pip install nemoguardrails

In [54]:
from nemoguardrails import RailsConfig
from nemoguardrails.integrations.langchain.runnable_rails import RunnableRails

prompt = ChatPromptTemplate.from_template("{question}")

config = RailsConfig.from_path("./config")
guardrails = RunnableRails(config)
guardrails.rails.register_action(complete_chain, "return_answer")

rails_chain = prompt | guardrails | StrOutputParser()


In [ ]:
response = rails_chain.invoke("Good Morning")
print(response)

Good morning! How can I assist you today?
